In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K


def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


smooth = 1e-15


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.models import Model

def build_vnet(input_shape, num_classes):
    inputs = Input(input_shape)
    conv1 = Conv2D(16, (5, 5), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, (5, 5), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (5, 5), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, (5, 5), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (5, 5), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same')(pool3)
    conv4 = concatenate([conv4, conv3])
    conv4 = Conv2D(128, (5, 5), activation='relu', padding='same')(conv4)
    conv4 = Conv2D(128, (5, 5), activation='relu', padding='same')(conv4)
    conv5 = Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same')(conv4)
    conv5 = concatenate([conv5, conv2])
    conv5 = Conv2D(64, (5, 5), activation='relu', padding='same')(conv5)
    conv5 = Conv2D(64, (5, 5), activation='relu', padding='same')(conv5)
    conv6 = Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same')(conv5)
    conv6 = concatenate([conv6, conv1])
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv6)
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv6)
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(conv6)

    model = Model(inputs, outputs)
    return model

input_shape = (256, 256, 3)
num_classes = 2  # Tumor and background classes
model = build_vnet( input_shape , num_classes)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  1216        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 32  12832       ['conv2d[0][0]']                 
                                )                                                             

In [4]:
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from glob import glob
import cv2
import numpy as np
import os

H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = tf.compat.as_str_any(x)  # Convert tensor to string
        y = tf.compat.as_str_any(y)  # Convert tensor to string
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    size = int(len(images) * split)
    train_x, valid_x = train_test_split(
        images, test_size=size, random_state=42)
    train_y, valid_y = train_test_split(
        masks, test_size=size, random_state=42)
    train_x, test_x = train_test_split(
        train_x, test_size=size, random_state=42)
    train_y, test_y = train_test_split(
        train_y, test_size=size, random_state=42)
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("VNet")

    """ Hyperparameters """
    batch_size = 2
    lr = 1e-4  # 0.0001
    num_epochs = 20
    model_path = "VNet/model.h5"
    csv_path = "VNet/data.csv"

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    train_x, train_y = shuffle(train_x, train_y)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    train_steps = (len(train_x)//batch_size)
    valid_steps = (len(valid_x)//batch_size)

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    input_shape = (256, 256, 3)
    num_classes = 1  # Modified to 1 class
    model = build_vnet(input_shape, num_classes)
    metrics = [tf.keras.metrics.BinaryAccuracy(), Recall(), Precision()]
    model.compile(loss="binary_crossentropy",
                  optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                          patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )


Train: 1840 - 1840
Valid: 612 - 612
Test: 612 - 612
Epoch 1/20
920/920 [==============================] - ETA: 0s - loss: 0.0887 - binary_accuracy: 0.9803 - recall_1: 0.0042 - precision_1: 0.0326
Epoch 1: val_loss improved from inf to 0.06516, saving model to VNet\model.h5
920/920 [==============================] - 533s 577ms/step - loss: 0.0887 - binary_accuracy: 0.9803 - recall_1: 0.0042 - precision_1: 0.0326 - val_loss: 0.0652 - val_binary_accuracy: 0.9823 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/20
920/920 [==============================] - ETA: 0s - loss: 0.0624 - binary_accuracy: 0.9826 - recall_1: 0.0096 - precision_1: 0.7666
Epoch 2: val_loss improved from 0.06516 to 0.05317, saving model to VNet\model.h5
920/920 [==============================] - 532s 579ms/step - loss: 0.0624 - binary_accuracy: 0.9826 - recall_1: 0.0096 - precision_1: 0.7666 - val_loss: 0.0532 - val_binary_accuracy: 0.9837 - val_recall_1: 0.0987 - val_precision_1: 0.86

In [5]:
from sklearn.model_selection import train_test_split
def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    size = int(len(images) * split)
    train_x, valid_x = train_test_split(
        images, test_size=size, random_state=42)
    train_y, valid_y = train_test_split(
        masks, test_size=size, random_state=42)
    train_x, test_x = train_test_split(
        train_x, test_size=size, random_state=42)
    train_y, test_y = train_test_split(
        train_y, test_size=size, random_state=42)
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [6]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.utils import CustomObjectScope
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  # (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    # (256, 256) ----> (256, 256, 1)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3))*255

    ori_y = np.expand_dims(ori_y, axis=-1)  # (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  # (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)*255.0
    temp = y_pred.copy()
    # make temp image red
    temp[:, :, 0] = 0
    temp[:, :, 1] = 0
    
    overlay = cv2.addWeighted(ori_x, 0.5,temp.astype('uint8'), 0.5, 0)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred,line,overlay], axis=1)
    cv2.imwrite(save_path, cat_images)


if __name__ == "__main__":
    create_dir("VNet_Results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("VNet/model.h5")

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Predicion and metrics values"""
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]
        name = name.split("\\")[-1]

        """Reading the image and mask"""
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """Prediction"""
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"VNet_Results/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(
            y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value,
                     recall_value, precision_value])
        
        
        

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Acurracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving all the results"""
    df = pd.DataFrame(
        SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("VNet/score02.csv")


  2%|▏         | 15/612 [00:06<02:47,  3.56it/s]c:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  7%|▋         | 40/612 [00:13<02:41,  3.54it/s]c:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  7%|▋         | 43/612 [00:13<02:35,  3.65it/s]c:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. U

Acurracy: 0.99168
F1: 0.64586
Jaccard: 0.55558
Recall: 0.64941
Precision: 0.72011


In [7]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.utils import CustomObjectScope
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  # (1, 256, 256, 3)
    return ori_x, x


def save_result(ori_x, y_pred, save_path):
    line = np.ones((H, 10, 3))*255

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)*255.0
    temp = y_pred.copy()
    # make temp image red
    temp[:, :, 0] = 0
    temp[:, :, 1] = 0
    
    overlay = cv2.addWeighted(ori_x, 0.5,temp.astype('uint8'), 0.5, 0)

    cat_images = np.concatenate([ori_x, line, y_pred, line, overlay], axis=1)
    cv2.imwrite(save_path, cat_images)


if __name__ == "__main__":
    create_dir("VNet_Other_Dataset_Test_Results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("VNet/model.h5")

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/dataset/test"

    """ Prediction and metrics values """
    SCORE = []
    
    test_images = glob(os.path.join(dataset_path, "*.jpg"))

    for x in tqdm(test_images, total=len(test_images)):
        name = os.path.basename(x)

        """ Reading the image """
        ori_x, x = read_image(x)

        """ Prediction """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = os.path.join("VNet_Other_Dataset_Test_Results", name)
        save_result(ori_x, y_pred, save_path)

        """ Flattening the numpy arrays """
        y_pred = y_pred.flatten()

        SCORE.append([name] + y_pred.tolist())

    """ Saving all the results """
    df = pd.DataFrame(SCORE, columns=["Image"] + [f"Pixel_{i}" for i in range(H*W)])
    df.to_csv("results/score02.csv", index=False)


100%|██████████| 15/15 [00:03<00:00,  4.77it/s]


OSError: Cannot save file into a non-existent directory: 'results'

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.utils import CustomObjectScope
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  # (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    # (256, 256) ----> (256, 256, 1)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x


def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3))*255

    ori_y = np.expand_dims(ori_y, axis=-1)  # (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  # (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)*255.0
    temp = y_pred.copy()
    # make temp image red
    temp[:, :, 0] = 0
    temp[:, :, 1] = 0
    
    overlay = cv2.addWeighted(ori_x, 0.5,temp.astype('uint8'), 0.5, 0)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred,line,overlay], axis=1)
    cv2.imwrite(save_path, cat_images)


if __name__ == "__main__":
    create_dir("VNet_Augmented_Results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("VNet/model.h5")

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/Augmented_Data"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Predicion and metrics values"""
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]
        name = name.split("\\")[-1]

        """Reading the image and mask"""
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """Prediction"""
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"VNet_Augmented_Results/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(
            y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value,
                     recall_value, precision_value])
        
        
        

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Acurracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving all the results"""
    df = pd.DataFrame(
        SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("UNet/score02_VNet_Augment.csv")


  0%|          | 4/3676 [00:06<1:02:37,  1.02s/it]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|          | 19/3676 [00:10<15:35,  3.91it/s] C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|          | 29/3676 [00:12<15:16,  3.98it/s]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted sampl

Acurracy: 0.98384
F1: 0.60430
Jaccard: 0.51298
Recall: 0.61168
Precision: 0.67010
